In [ ]:
import ase
import numpy as np
import matplotlib.pyplot as plt
from ase.build import fcc100,add_adsorbate
from ase import Atoms
import ase.visualize as asevisual
from ase import data as asedata

from ase.calculators.eam import EAM
from ase.optimize import FIRE
from ase.neb import NEB
from ase.neb import NEBTools
from ase.io import Trajectory
from ase.vibrations import Vibrations

<h1> Demonstration<h1>
<h5>Useful page: https://wiki.fysik.dtu.dk/ase/ase/neb.html<h5>
<h2>Direct hopping of adatom on silver fcc(100) surface.<h2>
<h2>Step 1: Prepare initial and final state structures<h2>

In [ ]:
import os
try:    
    os.mkdir('demo_dump/')
except:
    pass
# ASE build in function to generate surface
fcc100_Pd = fcc100('Pd', (6,6,4), a=None, vacuum=10, orthogonal=True, periodic=True)
# Get the NN separation between Ag-Ag
NN_dist= asedata.reference_states[46]['a']/np.sqrt(2)

# Get a location of the adatom to put
adatom_loc_xy = fcc100_Pd.positions[fcc100_Pd.positions[:,2].argsort()[-1]]+ NN_dist*np.array([-3.5,-3.5,0])
# This is relative to the 'top' of the slab
adatom_loc_height= NN_dist/np.sqrt(2)
# Make an ase obj for the initial state
ini_ase=fcc100_Pd.copy()
add_adsorbate(ini_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Similrar to above, but with the adatom shifted to the next holo-site
adatom_loc_xy = adatom_loc_xy+NN_dist*np.array([1,0,0])
fin_ase=fcc100_Pd.copy()
add_adsorbate(fin_ase, 'Pd', adatom_loc_height, position=(adatom_loc_xy[0], adatom_loc_xy[1]))

# Sanity check if the structure is okay
asevisual.view([ini_ase,fin_ase])

<h2>Step 2: Optimize the structures!<h2>

In [ ]:
ini_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
ini_e_before= ini_ase.get_potential_energy()
dyn = FIRE(ini_ase)
dyn.run(fmax=0.05)
ini_e_after= ini_ase.get_potential_energy()
print(f'Structure Optimized Energy {ini_e_before} -> {ini_e_after}')

fin_ase.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
fin_e_before= fin_ase.get_potential_energy()
dyn = FIRE(fin_ase)
dyn.run(fmax=0.05)
fin_e_after= fin_ase.get_potential_energy()
print(f'Structure Optimized Energy {fin_e_before} -> {fin_e_after}')

<h2>Step 3: ci-NEB Time. Initalize the states of the band<h2>

In [ ]:
# Make a list for images of the band, the first one is the initial, last one is final
# The middle ones are initalized as a copy of the initial state (will be adjusted afterwards)
# Let's put 7 images in the band
Nimages = 7
# From ASE documentation
# Be sure to use the copy method (or similar) to create new instances of atoms within the list of images fed to the NEB.
# Do not use something like [initial for i in range(3)], as it will only create references to the original atoms object.
NEB_images = [ini_ase.copy() for item in np.arange(Nimages+1)]
NEB_images+= [fin_ase]
# Intiallize the NEB object
neb=NEB(NEB_images, k=0.1, climb=True, method='improvedtangent')
# Adjust the images to an interpolation of positions between inital and final states
# IDPP: ‘Improved initial guess for minimum energy path calculations.’, S. Smidstrup, A. Pedersen, K. Stokbro and H. Jonsson, J. Chem. Phys. 140, 214106 (2014)
neb.interpolate('idpp',mic=True)
# Each bead has its own ASE calculator
for bead in NEB_images:
    bead.set_calculator(EAM(elements=['Pd'],potential='Pd_u3.eam'))
# Take a look at it first
asevisual.view(NEB_images)

<h2>Step 4: Really ci-NEB Time<h2>

In [ ]:
# Make a trajectory file, contains the images of each NEB iteration
dyn=FIRE(neb, trajectory='demo_dump/NEB.traj', logfile='demo_dump/neb.log')
# Make a trajectory file for each bead on the band for each NEB iteration
for bead_id in range(1,Nimages+1):
    traj = Trajectory('demo_dump/neb-%d.traj' % bead_id, 'w', NEB_images[bead_id])
    dyn.attach(traj)
# Run ciNEB
dyn.run(fmax=0.05)

<h2>Step 5: Extracting Stuffs<h2>

In [ ]:
# This gives the forward barrier
# just the energy difference of transition state and initial
nebtools = NEBTools(NEB_images)
barrier= nebtools.get_barrier(fit=False)
print(f'The barrier of forward reaction is {barrier[0]}eV')

In [ ]:
# Alternate way, calculate by yourself
nebtools = NEBTools(NEB_images)
IS_E= NEB_images[0].get_potential_energy()
TS_E= nebtools.get_barrier(fit=False,raw=True)
FS_E= NEB_images[-1].get_potential_energy()

Eact_IT = TS_E[0]-IS_E
Eact_FT = TS_E[0]-FS_E
print(f'The barrier of forward reaction is {Eact_IT:.4f} eV')
print(f'The barrier of backward reaction is {Eact_FT:.4f} eV')

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
plt.show()
plt.close()

In [ ]:
h = 4.135667689e-15
# No one wants to solve the whole Hessian involving all atoms (most of them are not moving)
indices = [144]
# Solve the vibrational energy for ini state
vib_IS = Vibrations(NEB_images[0],indices=indices,name="demo_dump/vib_IS")
vib_IS.clean()
vib_IS.run()
# Get the vibrational energies
IS_energies = vib_IS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
IS_real = [i.real for i in IS_energies if i.real != 0]

# Solve the vibrational energy for transition state
vib_TS = Vibrations(NEB_images[4],indices=indices,name="demo_dump/vib_TS")
vib_TS.clean()
vib_TS.run()
# Get the vibrational energies
TS_energies = vib_TS.get_energies()
# Get a product of them (for calculation of prefactor with hTST)
# One of them is supposed to be imaginary (is it?)
TS_real = [i.real for i in TS_energies if i.real != 0]

# Solve the vibrational energy for fin state
vib_FS = Vibrations(NEB_images[-1],indices=indices,name="demo_dump/vib_FS")
vib_FS.clean()
vib_FS.run()
FS_energies = vib_FS.get_energies()
FS_real = [i.real for i in FS_energies if i.real != 0]

nu_IT = (np.prod(IS_real)/np.prod(TS_real))/h
nu_FT = (np.prod(FS_real)/np.prod(TS_real))/h
print(f'The prefactor of forward reaction is {nu_IT:.4e} Hz')
print(f'The prefactor of backward reaction is {nu_FT:.4e} Hz')